In [10]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import pandas as pd

CSV_COLUMN_NAMES = ['SepalLength', 'SpealWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

#grab the datasets

train_path = tf.keras.utils.get_file("iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file("iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

#convert the databases to the pandas dataframe

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)

train.head()

#get the species columns to be used as label

train_y = train.pop('Species')
test_y = test.pop('Species')
train.head()

train.shape

train_y.head()

def input_fn(features, labels, training=True, batch_size=256):
  #Convert the inputs to a dataset.
  dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

  #Shuffle and repeat if in training mode
  if training:
    dataset = dataset.shuffle(1000).repeat()

  return dataset.batch(batch_size)



